In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem.
It is important that you do this by following these steps:

1. Define parameters and variables
2. Define the objective function
3. Define the constraints

Please note that each answer is always dedicated to one step. Therefore, do not provide extra information than what is asked. The answers will be manually combined by the user to create the full model."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
USER_PROMPT_1 = """We are now at the first step."""
USER_PROMPT_2 = """We are now at the second step."""
USER_PROMPT_3 = """We are now at the third step."""
PROBLEM_DESCRIPTION = """A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 11:11:22.185010


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1)
]

response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Step 1: Define parameters and variables

Parameters:

- Let \( T \) be the total units required, which is 239,600,480.
- Let \( A_s \) and \( A_u \) be the setup and unit costs for vendor A, respectively. \( A_s = 3855.34 \) and \( A_u = 61.150 \).
- Let \( B_s \) and \( B_u \) be the setup and unit costs for vendor B, respectively. The setup costs for vendor B are piecewise functions based on the quantity ordered, as are the unit costs.
- Let \( C_s \) and \( C_u \) be the setup and unit costs for vendor C, respectively. \( C_s = 13,456.00 \) and \( C_u = 62.019 \).
- Let \( D_s \) and \( D_u \) be the setup and unit costs for vendor D, respectively. \( D_s = 6,583.98 \) and \( D_u = 72.488 \).
- Let \( E_s \) and \( E_u \) be the setup and unit costs for vendor E, respectively. The setup and unit costs for vendor E are piecewise functions based on the quantity ordered.
- Let \( S_A, S_B, S_C, S_D, S_E \) be the maximum supply capacities for vendors A, B, C, D, and E, respectively. \( S_A = 33,000,000 \), \( S_B = 160,000,000 \), \( S_C = 165,600,000 \), \( S_D = 12,000,000 \), and \( S_E = 77,000,000 \). Note that for vendor B, zero units may be purchased.

Variables:

- Let \( x_A, x_B, x_C, x_D, x_E \) be the number of units ordered from vendors A, B, C, D, and E, respectively.

Now, proceed to Step 2.

In [6]:
print(response_1_text)

Step 1: Define parameters and variables

Parameters:

- Let \( T \) be the total units required, which is 239,600,480.
- Let \( A_s \) and \( A_u \) be the setup and unit costs for vendor A, respectively. \( A_s = 3855.34 \) and \( A_u = 61.150 \).
- Let \( B_s \) and \( B_u \) be the setup and unit costs for vendor B, respectively. The setup costs for vendor B are piecewise functions based on the quantity ordered, as are the unit costs.
- Let \( C_s \) and \( C_u \) be the setup and unit costs for vendor C, respectively. \( C_s = 13,456.00 \) and \( C_u = 62.019 \).
- Let \( D_s \) and \( D_u \) be the setup and unit costs for vendor D, respectively. \( D_s = 6,583.98 \) and \( D_u = 72.488 \).
- Let \( E_s \) and \( E_u \) be the setup and unit costs for vendor E, respectively. The setup and unit costs for vendor E are piecewise functions based on the quantity ordered.
- Let \( S_A, S_B, S_C, S_D, S_E \) be the maximum supply capacities for vendors A, B, C, D, and E, respectively. \(

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
]

response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [8]:
Markdown(response_2_text)

Step 2: Define the objective function

The objective is to minimize the total cost, which is the sum of the setup costs and the variable costs based on the unit costs for each vendor. Let \( z \) be the total cost. Then, the objective function is:

\[ z = A_s \cdot \delta_A + A_u \cdot \frac{x_A}{1000} + B_s \cdot \delta_B + B_u \cdot \frac{x_B}{1000} + C_s \cdot \delta_C + C_u \cdot \frac{x_C}{1000} + D_s \cdot \delta_D + D_u \cdot \frac{x_D}{1000} + E_s \cdot \delta_E + E_u \cdot \frac{x_E}{1000} \]

where \( \delta_A, \delta_B, \delta_C, \delta_D, \delta_E \) are binary variables indicating if a vendor is used or not. If a vendor is used, \( \delta = 1 \), otherwise \( \delta = 0 \). The unit costs and setup costs are multiplied by the appropriate constants to convert from thousands to units and from per thousand units to per unit.

Now proceed to Step 3.

In [9]:
print(response_2_text)

Step 2: Define the objective function

The objective is to minimize the total cost, which is the sum of the setup costs and the variable costs based on the unit costs for each vendor. Let \( z \) be the total cost. Then, the objective function is:

\[ z = A_s \cdot \delta_A + A_u \cdot \frac{x_A}{1000} + B_s \cdot \delta_B + B_u \cdot \frac{x_B}{1000} + C_s \cdot \delta_C + C_u \cdot \frac{x_C}{1000} + D_s \cdot \delta_D + D_u \cdot \frac{x_D}{1000} + E_s \cdot \delta_E + E_u \cdot \frac{x_E}{1000} \]

where \( \delta_A, \delta_B, \delta_C, \delta_D, \delta_E \) are binary variables indicating if a vendor is used or not. If a vendor is used, \( \delta = 1 \), otherwise \( \delta = 0 \). The unit costs and setup costs are multiplied by the appropriate constants to convert from thousands to units and from per thousand units to per unit.

Now proceed to Step 3.


In [10]:
messages_3 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
    ChatMessage(role="assistant", content=response_2_text),
    ChatMessage(role="user", content=USER_PROMPT_3),
]

response_3 = client.chat(
    model=MODEL_ID,
    messages=messages_3,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_3_text = response_3.choices[0].message.content

In [11]:
Markdown(response_3_text)

Step 3: Define the constraints

1. Total demand constraint: The total number of units ordered from all vendors must equal the total demand.

\[ x_A + x_B + x_C + x_D + x_E = T \]

2. Maximum supply capacity constraints: The number of units ordered from each vendor cannot exceed the vendor's maximum supply capacity.

\[ x_A \leq S_A, \quad x_B \leq S_B \cdot \delta_B, \quad x_C \leq S_C, \quad x_D \leq S_D, \quad x_E \leq S_E \]

3. Vendor B minimum order constraint: If vendor B is used, the number of units ordered must be at least 22,000,000.

\[ x_B \geq 22,000,000 \cdot \delta_B \]

4. Vendor E setup cost constraint: If more than 42,000,000 units are purchased from vendor E, an additional setup cost is incurred.

\[ E_s = \begin{cases} 0 & \text{if } x_E \leq 42,000,000 \\ 84,000 & \text{otherwise} \end{cases} \]

5. Vendor E unit cost constraint: If more than 42,000,000 units are purchased from vendor E, the unit cost decreases by 0.05% for every one million units ordered, up to 77,000,000 units.

\[ E_u = \begin{cases} 70.150 & \text{if } x_E \leq 42,000,000 \\ 68.150 - 0.05 \cdot \left\lfloor \frac{x_E - 42,000,000}{1,000,000} \right\rfloor & \text{if } 42,000,000 < x_E \leq 77,000,000 \\ \text{N/A} & \text{if } x_E > 77,000,000 \end{cases} \]

6. Non-negativity and integrality constraints: All variables must be non-negative, and the binary variables must be integer.

\[ x_A, x_B, x_C, x_D, x_E \geq 0, \quad x_A, x_B, x_C, x_D, x_E \in \mathbb{Z}, \quad \delta_A, \delta_B, \delta_C, \delta_D, \delta_E \in \{0, 1\} \]

Now, the optimization model is complete. It can be solved using a mixed-integer linear programming (MILP) solver to find the optimal solution for the number of units to order from each vendor and the resulting minimum total cost.

In [12]:
print(response_3_text)

Step 3: Define the constraints

1. Total demand constraint: The total number of units ordered from all vendors must equal the total demand.

\[ x_A + x_B + x_C + x_D + x_E = T \]

2. Maximum supply capacity constraints: The number of units ordered from each vendor cannot exceed the vendor's maximum supply capacity.

\[ x_A \leq S_A, \quad x_B \leq S_B \cdot \delta_B, \quad x_C \leq S_C, \quad x_D \leq S_D, \quad x_E \leq S_E \]

3. Vendor B minimum order constraint: If vendor B is used, the number of units ordered must be at least 22,000,000.

\[ x_B \geq 22,000,000 \cdot \delta_B \]

4. Vendor E setup cost constraint: If more than 42,000,000 units are purchased from vendor E, an additional setup cost is incurred.

\[ E_s = \begin{cases} 0 & \text{if } x_E \leq 42,000,000 \\ 84,000 & \text{otherwise} \end{cases} \]

5. Vendor E unit cost constraint: If more than 42,000,000 units are purchased from vendor E, the unit cost decreases by 0.05% for every one million units ordered, up to 77,

In [13]:
response_text = '\n'.join([response_1_text, response_2_text, response_3_text])

Markdown(response_text)

Step 1: Define parameters and variables

Parameters:

- Let \( T \) be the total units required, which is 239,600,480.
- Let \( A_s \) and \( A_u \) be the setup and unit costs for vendor A, respectively. \( A_s = 3855.34 \) and \( A_u = 61.150 \).
- Let \( B_s \) and \( B_u \) be the setup and unit costs for vendor B, respectively. The setup costs for vendor B are piecewise functions based on the quantity ordered, as are the unit costs.
- Let \( C_s \) and \( C_u \) be the setup and unit costs for vendor C, respectively. \( C_s = 13,456.00 \) and \( C_u = 62.019 \).
- Let \( D_s \) and \( D_u \) be the setup and unit costs for vendor D, respectively. \( D_s = 6,583.98 \) and \( D_u = 72.488 \).
- Let \( E_s \) and \( E_u \) be the setup and unit costs for vendor E, respectively. The setup and unit costs for vendor E are piecewise functions based on the quantity ordered.
- Let \( S_A, S_B, S_C, S_D, S_E \) be the maximum supply capacities for vendors A, B, C, D, and E, respectively. \( S_A = 33,000,000 \), \( S_B = 160,000,000 \), \( S_C = 165,600,000 \), \( S_D = 12,000,000 \), and \( S_E = 77,000,000 \). Note that for vendor B, zero units may be purchased.

Variables:

- Let \( x_A, x_B, x_C, x_D, x_E \) be the number of units ordered from vendors A, B, C, D, and E, respectively.

Now, proceed to Step 2.
Step 2: Define the objective function

The objective is to minimize the total cost, which is the sum of the setup costs and the variable costs based on the unit costs for each vendor. Let \( z \) be the total cost. Then, the objective function is:

\[ z = A_s \cdot \delta_A + A_u \cdot \frac{x_A}{1000} + B_s \cdot \delta_B + B_u \cdot \frac{x_B}{1000} + C_s \cdot \delta_C + C_u \cdot \frac{x_C}{1000} + D_s \cdot \delta_D + D_u \cdot \frac{x_D}{1000} + E_s \cdot \delta_E + E_u \cdot \frac{x_E}{1000} \]

where \( \delta_A, \delta_B, \delta_C, \delta_D, \delta_E \) are binary variables indicating if a vendor is used or not. If a vendor is used, \( \delta = 1 \), otherwise \( \delta = 0 \). The unit costs and setup costs are multiplied by the appropriate constants to convert from thousands to units and from per thousand units to per unit.

Now proceed to Step 3.
Step 3: Define the constraints

1. Total demand constraint: The total number of units ordered from all vendors must equal the total demand.

\[ x_A + x_B + x_C + x_D + x_E = T \]

2. Maximum supply capacity constraints: The number of units ordered from each vendor cannot exceed the vendor's maximum supply capacity.

\[ x_A \leq S_A, \quad x_B \leq S_B \cdot \delta_B, \quad x_C \leq S_C, \quad x_D \leq S_D, \quad x_E \leq S_E \]

3. Vendor B minimum order constraint: If vendor B is used, the number of units ordered must be at least 22,000,000.

\[ x_B \geq 22,000,000 \cdot \delta_B \]

4. Vendor E setup cost constraint: If more than 42,000,000 units are purchased from vendor E, an additional setup cost is incurred.

\[ E_s = \begin{cases} 0 & \text{if } x_E \leq 42,000,000 \\ 84,000 & \text{otherwise} \end{cases} \]

5. Vendor E unit cost constraint: If more than 42,000,000 units are purchased from vendor E, the unit cost decreases by 0.05% for every one million units ordered, up to 77,000,000 units.

\[ E_u = \begin{cases} 70.150 & \text{if } x_E \leq 42,000,000 \\ 68.150 - 0.05 \cdot \left\lfloor \frac{x_E - 42,000,000}{1,000,000} \right\rfloor & \text{if } 42,000,000 < x_E \leq 77,000,000 \\ \text{N/A} & \text{if } x_E > 77,000,000 \end{cases} \]

6. Non-negativity and integrality constraints: All variables must be non-negative, and the binary variables must be integer.

\[ x_A, x_B, x_C, x_D, x_E \geq 0, \quad x_A, x_B, x_C, x_D, x_E \in \mathbb{Z}, \quad \delta_A, \delta_B, \delta_C, \delta_D, \delta_E \in \{0, 1\} \]

Now, the optimization model is complete. It can be solved using a mixed-integer linear programming (MILP) solver to find the optimal solution for the number of units to order from each vendor and the resulting minimum total cost.

In [14]:
print(response_text)

Step 1: Define parameters and variables

Parameters:

- Let \( T \) be the total units required, which is 239,600,480.
- Let \( A_s \) and \( A_u \) be the setup and unit costs for vendor A, respectively. \( A_s = 3855.34 \) and \( A_u = 61.150 \).
- Let \( B_s \) and \( B_u \) be the setup and unit costs for vendor B, respectively. The setup costs for vendor B are piecewise functions based on the quantity ordered, as are the unit costs.
- Let \( C_s \) and \( C_u \) be the setup and unit costs for vendor C, respectively. \( C_s = 13,456.00 \) and \( C_u = 62.019 \).
- Let \( D_s \) and \( D_u \) be the setup and unit costs for vendor D, respectively. \( D_s = 6,583.98 \) and \( D_u = 72.488 \).
- Let \( E_s \) and \( E_u \) be the setup and unit costs for vendor E, respectively. The setup and unit costs for vendor E are piecewise functions based on the quantity ordered.
- Let \( S_A, S_B, S_C, S_D, S_E \) be the maximum supply capacities for vendors A, B, C, D, and E, respectively. \(

## Step 2 - Generate the Pyomo Code

In [15]:
messages_4 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_text)
]

In [16]:
response_4 = client.chat(
    model=MODEL_ID,
    messages=messages_4,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_4_text = response_4.choices[0].message.content

In [17]:
Markdown(response_4_text)

To solve this optimization problem using Pyomo, we first need to install the necessary packages. If you haven't installed Pyomo and a solver like GLPK, you can do so by running the following commands:

```bash
pip install pyomo
pip install pyomo.solvers
pip install pyomo.environ
pip install pyomo.gdp
pip install pyomo.glpk
```

Now, let's write the Pyomo code for the given optimization problem. I will use the provided sample data and define the model as described in the problem statement.

```python
import pyomo.environ as pyo

# Step 1: Define parameters and variables
T = 239600480  # Total units required
A_s = 3855.34  # Setup cost for vendor A
A_u = 61.150  # Unit cost for vendor A
B_s = {0: 0, 22000000: 14900.00, 160000000: 16900.00}  # Setup cost for vendor B
B_u = {0: 0, 22000000: 62.505, 160000000: 61.150}  # Unit cost for vendor B
C_s = 13456.00  # Setup cost for vendor C
C_u = 62.019  # Unit cost for vendor C
D_s = 6583.98  # Setup cost for vendor D
D_u = 72.488  # Unit cost for vendor D
E_s = {0: 0, 42000000: 0, 77000000: 84000.00}  # Setup cost for vendor E
E_u = {0: 0, 42000000: 70.150, 77000000: 68.150}  # Unit cost for vendor E
S_A = 33000000  # Max supply capacity for vendor A
S_B = 160000000  # Max supply capacity for vendor B
S_C = 165600000  # Max supply capacity for vendor C
S_D = 12000000  # Max supply capacity for vendor D
S_E = 77000000  # Max supply capacity for vendor E

# Step 2: Define the Pyomo model
model = pyo.ConcreteModel()

# Define variables
model.x_A = pyo.Var(within=pyo.NonNegativeIntegers)
model.x_B = pyo.Var(within=pyo.NonNegativeIntegers)
model.x_C = pyo.Var(within=pyo.NonNegativeIntegers)
model.x_D = pyo.Var(within=pyo.NonNegativeIntegers)
model.x_E = pyo.Var(within=pyo.NonNegativeIntegers)
model.delta_A = pyo.Var(within=pyo.Binary)
model.delta_B = pyo.Var(within=pyo.Binary)
model.delta_C = pyo.Var(within=pyo.Binary)
model.delta_D = pyo.Var(within=pyo.Binary)
model.delta_E = pyo.Var(within=pyo.Binary)

# Step 3: Define the objective function
def obj_rule(model):
    return (
        A_s * model.delta_A
        + A_u * model.x_A / 1000
        + B_s[model.x_B] * model.delta_B
        + B_u[model.x_B] * model.x_B / 1000
        + C_s * model.delta_C
        + C_u * model.x_C / 1000
        + D_s * model.delta_D
        + D_u * model.x_D / 1000
        + E_s[model.x_E] * model.delta_E
        + E_u[model.x_E] * model.x_E / 1000
    )

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Step 4: Define the constraints
def demand_constraint_rule(model):
    return model.x_A + model.x_B + model.x_C + model.x_D + model.x_E == T

model.demand_constraint = pyo.Constraint(rule=demand_constraint_rule)

def supply_capacity_constraint_rule_A(model):
    return model.x_A <= S_A

model.supply_capacity_constraint_A = pyo.Constraint(
    rule=supply_capacity_constraint_rule_A
)

def supply_capacity_constraint_rule_B(model):
    return model.x_B <= S_B * model.delta_B

model.supply_capacity_constraint_B = pyo.Constraint(
```
...

I've written the Pyomo code for the given optimization problem with the provided sample data. However, it's important to note that the piecewise functions for vendor B, E's setup cost, and unit cost need to be handled carefully in Pyomo. I've provided a starting point for the code, but you may need to adjust the piecewise functions according to the Pyomo documentation.

Once the code is complete, you can solve the model using a solver like GLPK by adding the following lines at the end of the code:

```python
# Step 5: Solve the model
solver = pyo.SolverFactory("glpk")
results = solver.solve(model)

# Step 6: Print the results
print("Optimal Solution:")
print(f"x_A: {pyo.value(model.x_A)}")
print(f"x_B: {pyo.value(model.x_B)}")
print(f"x_C: {pyo.value(model.x_C)}")
print(f"x_D: {pyo.value(model.x_D)}")
print(f"x_E: {pyo.value(model.x_E)}")
print(f"Total Cost: {pyo.value(model.obj)}")
```

This will print the optimal solution for the number of units to order from each vendor and the resulting minimum total cost.

In [18]:
print(response_4_text)

To solve this optimization problem using Pyomo, we first need to install the necessary packages. If you haven't installed Pyomo and a solver like GLPK, you can do so by running the following commands:

```bash
pip install pyomo
pip install pyomo.solvers
pip install pyomo.environ
pip install pyomo.gdp
pip install pyomo.glpk
```

Now, let's write the Pyomo code for the given optimization problem. I will use the provided sample data and define the model as described in the problem statement.

```python
import pyomo.environ as pyo

# Step 1: Define parameters and variables
T = 239600480  # Total units required
A_s = 3855.34  # Setup cost for vendor A
A_u = 61.150  # Unit cost for vendor A
B_s = {0: 0, 22000000: 14900.00, 160000000: 16900.00}  # Setup cost for vendor B
B_u = {0: 0, 22000000: 62.505, 160000000: 61.150}  # Unit cost for vendor B
C_s = 13456.00  # Setup cost for vendor C
C_u = 62.019  # Unit cost for vendor C
D_s = 6583.98  # Setup cost for vendor D
D_u = 72.488  # Unit cost f

### Code Executability

### Solution Correctness